# Rysowanie do PDF

In [1]:
import numpy as np
import pandas as pd
import random
from reportlab.pdfgen import canvas
import copy

import generating_cathegories_functions as funs
from puzzle_class import puzzle
from pdf_printing_functions import rysuj_zagadke

### Wylosuj i wydrukuj zagadkę

In [19]:
seed

12131

In [2]:
seed = 12131#random.randint(100,100000)
puzzle1 = puzzle(4,5)
puzzle1.set_seed(seed)
puzzle1.draw_cathegories(diff=3)
puzzle1.draw_clues()

print(puzzle1.is_grid_completed(), puzzle1.is_grid_contradictory())


True False


In [5]:
print(len(puzzle1.clues))
puzzle1.try_to_restrict_clues()      
print(len(puzzle1.clues))
puzzle1.try_to_solve()        
if puzzle1.contradictory:
    print("Wyszło sprzeczne!")
else:
    c = canvas.Canvas("p"+str(puzzle1.seed)+"K"+str(puzzle1.K)+"k"+str(puzzle1.k)+"c"+str(len(puzzle1.clues))+".pdf")
    rysuj_zagadke(puzzle1, c)
    c.showPage()
    c.save()

10
[]
10


# Nowe

In [23]:
puzzle1 = puzzle(4,5)
puzzle1.set_seed(55)
puzzle1.draw_cathegories()
puzzle1.cathegories

[('cathegorical', ['królowa', 'pastuszek', 'dinozaur', 'myszka', 'kowboj']),
 ('cathegorical', ['Wu', 'Di', 'Huang', 'Mao', 'Zhang']),
 ('numerical',
  [15.0, 17.5, 20.0, 22.5, 25.0],
  {'x=y+2.5', 'x=y+5.0', 'x=y+7.5'},
  'Wartość stałej'),
 ('numerical',
  [2.0, 6.0, 18.0, 54.0, 162.0],
  {'x=y*27', 'x=y*3', 'x=y*9'},
  '@ klocków')]

In [24]:
for i in range(50):
    puzzle1.clear_grid()
    puzzle1.clues = []
    puzzle1.draw_clues()
    print(puzzle1.clues[0])
    print(i)
    if puzzle1.is_grid_completed() and not puzzle1.is_grid_contradictory():
        break

{'typ': 2, 'K1': 3, 'i1': 2, 'K2': 0, 'i2': 4, 'K3': 0, 'i3': 1, 'K6': 2}
0
{'typ': 2, 'K1': 2, 'i1': 0, 'K2': 2, 'i2': 4, 'K3': 0, 'i3': 4, 'K6': 3}
1
{'typ': 2, 'K1': 0, 'i1': 2, 'K2': 1, 'i2': 4, 'K3': 0, 'i3': 1, 'K6': 2}
2
{'typ': 3, 'K1': 0, 'i1': 0, 'K2': 0, 'i2': 1, 'K6': 3, 'diff': 9.0, 'oper': '*'}
3


In [25]:
puzzle1.print_info()

Completed: True, Contradictory: False


In [26]:
print(len(puzzle1.clues))
puzzle1.try_to_restrict_clues()      
print(len(puzzle1.clues))

11
[5]
10


In [27]:
puzzle1.try_to_solve()
puzzle1.print_info()

Completed: True, Contradictory: False


In [28]:
#c = canvas.Canvas("testy.pdf")
c = canvas.Canvas("p"+str(puzzle1.seed)+"K"+str(puzzle1.K)+"k"+str(puzzle1.k)+"c"+str(len(puzzle1.clues))+".pdf")
rysuj_zagadke(puzzle1, c)
c.showPage()
c.save()

# Śmietnik

In [2]:
correct = 0
N = 100
for i in range(N):
    seed = random.randint(100,100000)
    puzzle1 = puzzle(4,5)
    puzzle1.set_seed(seed)
    puzzle1.draw_cathegories(diff=3)
    puzzle1.draw_clues()
    if puzzle1.is_grid_completed() and not puzzle1.is_grid_contradictory():
        correct += 1

#print(puzzle1.is_grid_completed(), puzzle1.is_grid_contradictory())

Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
Puzzle is contradictory with clue 2!
P

Exception: Nie udało się znaleźć wystarczającej liczby interpretacji dla zmiennych numerycznych!

In [90]:
seed = random.randint(100,100000)
puzzle1 = puzzle(3,6)
puzzle1.set_seed(seed)
puzzle1.draw_cathegories(diff=3)
puzzle1.draw_clues()

print(puzzle1.is_grid_completed(), puzzle1.is_grid_contradictory())
print(len(puzzle1.clues))
puzzle1.try_to_restrict_clues()      
print(len(puzzle1.clues))
puzzle1.try_to_solve()        
if puzzle1.is_grid_contradictory():
    print("Wyszło sprzeczne!")
else:
    c = canvas.Canvas("p"+str(puzzle1.seed)+"K"+str(puzzle1.K)+"k"+str(puzzle1.k)+"c"+str(len(puzzle1.clues))+".pdf")
    rysuj_zagadke(puzzle1, c)
    c.showPage()
    c.save()

True False
10
[]
10


# Inne

In [67]:
def fonts(canvas):
    from reportlab.lib.units import inch
    for font in canvas.getAvailableFonts():
        print(font)
fonts(c)

Courier
Courier-Bold
Courier-BoldOblique
Courier-Oblique
Helvetica
Helvetica-Bold
Helvetica-BoldOblique
Helvetica-Oblique
Symbol
Times-Bold
Times-BoldItalic
Times-Italic
Times-Roman
ZapfDingbats
sans-serif
